# Interactive Seed Counting Workflow 

## Part 1: Workflow setup

In [ ]:
# Import plantcv 
from plantcv import plantcv as pcv
# Set the debugging parameter to plot 
pcv.params.debug = "plot"

In [ ]:
# Set the notebook display method
# inline = embedded plots, widget = interactive plots
%matplotlib inline

In [ ]:
# Updated text params so that labels are readable 
pcv.params.text_size = 50
pcv.params.text_thickness = 35

In [ ]:
# Change plot size to be larger to better see each colorspace 
import matplotlib
matplotlib.rcParams["figure.figsize"] = [8,8]

In [ ]:
# Read image

# Inputs:
#   filename - Image file to be read in 
#   mode - How to read in the image; either 'native' (default), 'rgb', 'gray', or 'csv'
img, path, filename = pcv.readimage(filename="imgs/camelina_seeds.jpg")


## Part 1: Create seed mask 

#### Visualize colorspaces

The visualization tool converts the color image into HSV and LAB colorspaces and displays the grayscale channels in a matrix so that they can be visualized simultaneously. The idea is to select a channel that maximizes the difference between the plant and the background pixels.

In [ ]:
# Visualize all colorspaces to help determine a thresholding channel 
# Inputs:
#   rbg_img      = original image
#   original_img = whether to include the original RGB images in the display: True (default) or False
colorspaces = pcv.visualize.colorspaces(img)

#### Convert the color image to grayscale

Converts the input color image into the LAB colorspace and returns the l (lightness) channel as a grayscale image.

In [ ]:
# Inputs:
#   rbg_img - original image
#   channel - desired colorspace ('l', 'a', or 'b')
l = pcv.rgb2gray_lab(rgb_img=img, channel="l")

In [ ]:
# Inputs:
#   img         = gray image in selected colorspace
#   mask        = None (default), or mask
#   bins        = 100 (default) or number of desired number of evenly spaced bins
#   lower-bound = None (default) or minimum value on x-axis
#   upper-bound = None (default) or maximum value on x-axis
#   title       = None (default) or custom plot title
#   hist_data   = False (default) or True (if frequency distribution data is desired)
hist = pcv.visualize.histogram(img=l)

#### Threshold the grayscale image

Use a threshold function (binary in this case) to segment the grayscale image into plant (white) and background (black) pixels. Using the histogram above, a threshold point between 120-125 will segment the plant and background peaks. Because the plant has darker pixels in this image, use object_type="dark" to do a traditional threshold.

In [ ]:
l_thresh = pcv.threshold.binary(gray_img=l, threshold=120, max_value=255, object_type="dark")

#### Identify objects within image

From the binary mask, we can identify the object (plant) within the image, which can then be used later for analyses.

In [ ]:
# Inputs:
#   img  = rgb image
#   mask = binary mask
plant_obj, plant_hier = pcv.find_objects(img=img, mask=l_thresh)

#### Identify objects within a region of interest (ROI)
To eventually combine all of the objects into a singular object that identifies the plant, we need to identify a region of interest (ROI) which will either fully encapsulate or overlap with plant material. This way, if objects are identified due to "salt" noise or other background elements, they will be filtered out. In this case, a rectangular ROI that partially overlaps with the plant object can be used to filter out some of the excess noise around the plant.

In [ ]:
# Inputs:
#   img = image
#   x   = leftmost x coordinate of ROI
#   y   = topmost y coordinate of ROI
#   h   = height of ROI
#   w   = width of ROI
roi_contour, roi_hierarchy = pcv.roi.rectangle(img=img, x=0, y=900, h=2168, w=3500)

In [ ]:
# Inputs:
#   img            = image
#   roi_type       = 'partial' to include objects that partially overlap with the ROI, 'cutto' to exclude any objects that 
#                     are not within the ROI, or 'largest' to keep the largest contour
#   roi_contour    = pre-defined region of interest
#   roi_hierarchy  = pre-defined roi hierarchy
#   object_contour = plant object
#   obj_hierarchy  = object hierarchy
objects, hier, seed_mask, obj_area = pcv.roi_objects(img=img, roi_contour=roi_contour, roi_hierarchy=roi_hierarchy,
                                                     object_contour=plant_obj, obj_hierarchy=plant_hier, roi_type='partial')

# Part 2: Detect disks 

In this case, detecting disks how to identify camilina seeds and label them in a way that can be annotated downstream in the workflow. 

In [ ]:
# Inputs:
#   bin_img        = binary mask image
#   ecc_thresh     = eccentricity threshold below which a region is detected
discs_mask, discs_coor = pcv.detect_discs(bin_img=seed_mask, ecc_thresh=0.9)

We can visualize the contents of the disc coordinates output from this function. 

In [ ]:
print(discs_coor)

# Part 3: Click count 

Using the `ClickCount` class from PlantCV we can interactively annotate the seeds identified in the disc detection step. 

In [ ]:
# Initialization
seed_counter = pcv.visualize.ClickCount(img=img)
# Import coordinates (if available, you can also populate all annotations by hand optionally)
seed_counter.import_coords(coords=discs_coor, label="total")

In [ ]:
# View "total" class and update annotations if desired 

seed_counter.view(label="total", color="c", view_all=True)

Notice one rogue object detected on the right hand side and two at the upper left that need to be deselected. No seeds are unaccounted for which is good! 

In [ ]:
# Save out coordinates of seeds 
seed_counter.save_coords(coord_file="example_seed_coords.json")